## Langchain demo of output parser

This exercise will cover output parsers, what they are, why we use them and some examples.
First, let's get all the config and imports stuff in order.

1. Notice the the below code block has `import config`, find the `config.py` file (in the same directory) and put your openai API key in here.  If you do not have one yet, create a new account and refer to [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key) to find your openai API key. 

2. Once you have your key, update the key in `config.py`, it should have the format of `sk-XXXXXXXXXXXXXXXXXX` (number of X's is not exact so don't take that literally please).

3. Run the block below once the above 2 are done so setup the environment.

In [13]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import HuggingFaceHub, LLMChain

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List
import config
import os

os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

# uncomment below if you want to use huggingface instead, when you run this, you will see an input box below, paste in your huggingface token in there
# and wait to see "........"
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

 ········


### What are output parsers?
We have seen that language models output text, but usually you'd like to get more structured information output than just text.  

Let's take an example:
Suppose you ask your llm to build you a car from a description, the output could be a long string such as `2020 Honda CR-V AWD (All Wheel Drive) ...`.  This is great, but how can we actually use this information?

Thinking from an OOP perspective, it would be better for us to ***model*** this output in an object so that it can interact with other stuff.  In other words, we would like some struct/class/object that might look something like this:

```
class Car:
    model
    make
    drivetrain (AWD, 4WD, RWD, ...)
    ...
```

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

"Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
"Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.
And then one optional one:

"Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt t

That's a lot of text to read, let's illustrate with some more examples with the main output parser, the `PydanticOutpu.o do so.

In [14]:
# Set up the model with temperature
# temperature is a measure of how creative we want the model to be
model_name = 'text-davinci-003'
temperature = 0.5
model = OpenAI(model_name=model_name, temperature=temperature)

# uncomment below to test you have credits, if not go back up and use huggingface
# name = model("say hi")

# uncomment below to use huggingface
repo_id = "google/flan-t5-xxl"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
hmodel = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": temperature, "max_length": 64}
)

In [15]:
# Define your desired data structure.
class Dish(BaseModel):
    name: str = Field(description="name of the dish")
    ingredients: List[str] = Field(description="list of ingredients required")

# Now to setup the use of the model
query = "Generate a recipe name, and list the ingredients required for the recipe"

# Notice we set the object to our Dish class
parser = PydanticOutputParser(pydantic_object=Dish)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


_input = prompt.format_prompt(query=query)
inputstr = _input.to_string()

# to use openai
output = model(inputstr)
parser.parse(output)

# uncomment below (and comment above block 2 lines) to use huggingface
# llm_chain = LLMChain(prompt=prompt, llm=hmodel)
# output = llm_chain.run(inputstr)
# print(output)
# parser.parse(output)



name="Spicy Tomato Soup" ingredients="tomatoes, onion, garlic


Below, make another Python class called ChickenJoke with the following properties:
setup: string -> the setup for the joke (Why did the chicken cross the road)
answer: string -> the resolution for the joke

Ask your llm to come up with a funny answer, and parse it into the ChickenJoke class.